# Data Workflow Lab 1

Clean and summarize Project 3 data.

### Learning Objectives

* Practice text cleaning techniques
* Practice datatype conversion
* Practice filling in missing values with either 0 or the average in the column
* Practice categorical data techniques
* Transform data into usable quantities


In [2]:
% matplotlib inline
import datetime
import numpy as np
import pandas as pd

In [14]:
# Load the data
df = pd.read_csv("Iowa_Liquor_sales_sample_10pct.csv")
print df.columns
df.head()

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')


,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [4]:
df.shape

(270955, 18)

## Clean the data

Let's practice our data cleaning skills on the Project 3 dataset. If you don't remember how to do any of these tasks, look back at your work from the previous weeks or search the internet. There are many blog articles and Stack Overflow posts that cover these topics.

You'll want to complete at least the following tasks:
* Remove redundant columns
* Remove "$" prices from characters and convert values to floats.
* Convert dates to pandas datetime objects
* Convert category floats to integers
* Drop or fill in bad values

In [15]:
# Remove redundant columns
df.drop(['County','Category Name','Vendor Number','Item Description','Bottle Volume (ml)','State Bottle Cost','Volume Sold (Gallons)'],axis=1,inplace=True)

In [16]:
# Remove $ from certain columns
dollar_cols = ['State Bottle Retail','Sale (Dollars)']
for col in dollar_cols:
    try:
        df[col] = [float(x[1:]) for x in df[col]]
    except Exception:
        print("Cannot convert %s" % col)

In [17]:
# Convert dates
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df.head()

,Date,Store Number,Zip Code,County Number,Category,Item Number,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters)
0,2015-11-04,3717,50674,9.0,1051100.0,54436,6.75,12,81.00,9.0
1,2016-03-02,2614,52807,82.0,1011100.0,27605,20.63,2,41.26,1.5
2,2016-02-11,2106,50613,7.0,1011200.0,19067,18.89,24,453.36,24.0
3,2016-02-03,2501,50010,85.0,1071100.0,59154,14.25,6,85.50,10.5
4,2015-08-18,3654,50421,99.0,1031080.0,35918,10.80,12,129.60,21.0


In [18]:
# Drop or replace bad values
df.apply(lambda x: sum(pd.isnull(x)))
df.dropna(subset=["County Number","Category"],inplace=True)
# Convert integers
df['County Number'] = [int(x) for x in df['County Number']]

## Filter the Data

Some stores may have opened or closed in 2015. These data points will heavily skew our models, so we need to filter them out or find a way to deal with them.

You'll need to provide a summary in your project report about these data points. You may also consider using the monthly sales in your model and including other information (number of months or days each store is open) in your data to handle these unusual cases.

Let's record the first and last sales dates for each store. We'll save this information for later when we fit our models.

In [8]:
# Determine which stores were open all of 2015
# Find the first and last sales date.
df2015 = df[df['Date'].dt.year<2016]
nb_days_open = df2015.pivot_table(index="Vendor Number",values="Date",aggfunc=lambda x: np.max(x)-np.min(x))
#print(nb_days_open)
# Filter out stores that opened or closed throughout the year
# You may want to save this step until you start modelling
nb_days_open=nb_days_open.reset_index()
vendor_open_all_year = [nb_days_open.ix[i,0] for i in range(1,len(nb_days_open.ix[:,0])) if nb_days_open.ix[i,1].days>300]

## Compute New Columns and Tables

Since we're trying to predict sales and/or profits, we'll want to compute some intermediate data. There are a lot of ways to do thisand good use of pandas is crucial. For example, for each transaction we may want to know:
* margin, retail cost minus bottle cost
* price per bottle
* price per liter

We'll need to make a new dataframe that indexes quantities by store:
* sales per store for all of 2015
* sales per store for Q1 2015
* sales per store for Q1 2016
* total volumes sold
* mean transaction revenue, gross margin, price per bottle, price per liter, etc.
* average sales per day
* number of days open

Make sure to retain other variables that we'll want to use to build our models, such as zip code, county number, city, etc. We recommend that you spend some time thinking about the model you may want to fit and computing enough of the suggested quantities to give you a few options.

Bonus tasks:
* Restrict your attention to stores that were open for all of 2015 and Q1 2016. Stores that opened or closed in 2015 will introduce outliers into your data.
* For each transaction we have the item category. You may be able to determine the store type (primarily wine, liquor, all types of alcohol, etc.) by the most common transaction category for each store. This could be a useful categorical variable for modelling. 

In [31]:
# Margin and Price per liter


In [48]:
# Sales per store, 2015
#df.reset_index(inplace=True)

# Filter by our start and end dates
df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales = df[mask]


# Group by store name
sales = sales.groupby(by=["Store Number"], as_index=False)

print(sales.head(2))


# Compute sums, means
sales = sales.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0], # just extract once, should be the same
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})
# Collapse the column indices
sales.columns = [' '.join(col).strip() for col in sales.columns.values]
# Rename columns

# Quick check
sales.head()

        level_0   index       Date  Store Number           City Zip Code  \
0             0   22111 2015-01-08          2106    CEDAR FALLS    50613   
1             1   47659 2015-01-08          2106    CEDAR FALLS    50613   
654         654   27649 2015-01-07          2113         GOWRIE    50543   
655         655   50536 2015-01-07          2113         GOWRIE    50543   
838         838    2246 2015-01-08          2130       WATERLOO    50703   
839         839   55391 2015-01-08          2130       WATERLOO    50703   
1347       1347   16908 2015-01-08          2152       ROCKWELL    50469   
1348       1348   68607 2015-01-08          2152       ROCKWELL    50469   
1508       1508    7458 2015-01-07          2178         WAUKON    52172   
1509       1509   26706 2015-01-07          2178         WAUKON    52172   
1803       1803    2791 2015-01-05          2190     DES MOINES    50314   
1804       1804   14630 2015-01-05          2190     DES MOINES    50314   
3429       3

KeyError: 'Price per Liter'

In [ ]:
# Q1 sales, may want to also use aggregate as above to have more columns (means, etc.)

# Sales 2015  Q1

# Sales 2016 Q1


Proceed with any calculations that you need for your models, such as grouping
sales by zip code, most common vendor number per store, etc. Once you have finished adding columns, be sure to save the dataframe.

In [ ]:
# Compute more things
# ...

In [ ]:
# Save this dataframe

# Let's add the dates computed above to this data.
sales["First Date"] = dates['Date amin']
sales["Last Date"] = dates['Date amax']

sales.to_csv("sales.csv")